[![Open in Layer](https://development.layer.co/assets/badge.svg)](https://app.layer.ai/layer/Recommendation_System_and_Product_Categorisation_Project/) [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/layerai/examples/blob/main/recommendation-system/Recommendation_System.ipynb) [![Layer Examples Github](https://badgen.net/badge/icon/github?icon=github&label)](https://github.com/layerai/examples/tree/main/recommendation-system)


In this e-commerce example walkthrough, we will develop and build a Recommendation System on  Layer. We will use a public clickstream dataset for this example project. For more information about the dataset, you could check out its Kaggle page here: https://www.kaggle.com/datasets/tunguz/clickstream-data-for-online-shopping

# Install Requirements

In [ ]:
!pip install layer --upgrade

In [ ]:
!rm -rf examples/recommendation-system
!git clone https://github.com/layerai/examples/

# Getting Started with Layer

Layer is an MLOps platform which advances ML pipelines with remote computation and tracking.

In [8]:
from layer.decorators import dataset, model,resources
from layer.client import Dataset, Model
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import layer

## Login to Layer

---



Let's login to Layer first.

In [ ]:
layer.login()

## Initialize Layer Project
Now we are ready to init our project. Layer Project is basically an ML Repo hosted on Layer where you can store your datasets, models, metrics

In [11]:
layer.init("Ecommerce_Recommendation_System", pip_packages=["gensim","sklearn","pandas","numpy"], fabric="f-medium")

Project(name='Recommendation_System_and_Product_Categorisation_Project', raw_datasets=[], derived_datasets=[], models=[], path=PosixPath('.'), project_files_hash='', readme='', account=Account(id=UUID('add1b570-c8e7-4187-b747-1d01104893a9'), name='layer'), _id=UUID('3335ab76-e638-4bb5-92c3-caabd647e298'), functions=[])

Your project is ready. Find your project here:

https://app.layer.ai

# **Data Transformation**

We will create total of 4 Layer datasets in this project. Here is the list of those datasets and their little descriptions.

*  **raw_session_based_clickstream_data:** This is basically identical to the source csv file which is a public Kaggle dataset: https://www.kaggle.com/datasets/tunguz/clickstream-data-for-online-shopping

    It is just Layer Dataset definition of the same clickstream raw data.

* **sequential_products:** This is a Layer dataset derived from the previous dataset which consists of sequences of products viewed in order per session. 


* **product_ids_and_vectors:** This is a Layer dataset which stores product vectors (embeddings) returned from Word2Vec algorithm.


* **final_product_clusters:** This is a Layer dataset which stores assigned cluster numbers per product and other members of those clusters.

Once you are done with building datasets, you can see them all here:

https://app.layer.ai/layer/Recommendation_System_and_Product_Categorisation_Project/datasets/

## **Model**

We will be training a K-Means model from sklearn. We will fit our clustering model using product vectors that we have created previously. You can find all the model experiments and logged data here:

Once you are done with training the model, you can see it here:

https://app.layer.ai/layer/Recommendation_System_and_Product_Categorisation_Project/models/


In [17]:
@resources("examples/recommendation-system/e-shop clothing 2008.csv")
@dataset("raw_session_based_clickstream_data")
def raw_session_based_clickstream_data():
  data = pd.read_csv("examples/recommendation-system/e-shop clothing 2008.csv",delimiter = ';')

  layer.log({"Dataset Description": "Raw clickstream data of an e-commerce clothing company"})

  return data

In [15]:
# remove consecutive duplicates from list
def remove_consec_duplicates(raw_lst):
  previous_value = None
  new_lst = []

  for elem in raw_lst:
    if elem != previous_value:
        new_lst.append(elem)
        previous_value = elem
        
  return new_lst


@dataset("sequential_products",dependencies=[Dataset('raw_session_based_clickstream_data')])
def generate_sequential_products():
  # Load dataset
  raw_clickstream = layer.get_dataset("raw_session_based_clickstream_data").to_pandas()

  # Rename columns
  data = raw_clickstream.rename(columns={"session ID": "session_id", "page 2 (clothing model)": "product_id"})
  # Remove sessions where only a single product is viewed
  data = data.groupby("session_id").filter(lambda x: len(x) > 1)
  # Group product view sequences in order by session id
  data = data.sort_values("order").groupby("session_id")["product_id"].apply(list)
  # Remove consecutive duplicate product views from the sequences genereated in the previous step
  data = data.apply(remove_consec_duplicates)

  #Convert series to data frame since you could only return dataframe in a Layer dataset decorator
  data = data.to_frame().reset_index().rename(columns={"product_id": "chronological_product_sequence"})

  layer.log({"Dataset Description": "Session-based sequences of products in view order"})

  return data

In [ ]:
@dataset("product_ids_and_vectors", dependencies=[Dataset("sequential_products")])
def create_product_embeddings():
  import gensim
  from gensim.models import Word2Vec
  # Fetch dataset from Layer
  session_based_product_sequences = layer.get_dataset("sequential_products").to_pandas()
  
  # Create Gensim CBOW model
  session_product_sequences = session_based_product_sequences['chronological_product_sequence'].apply(list)
  word2vec_model = gensim.models.Word2Vec(session_product_sequences, min_count = 1, size = 10, window = 5)
  
  # numpy.ndarrays of product vectors
  product_vectors = word2vec_model.wv.vectors

  productID_list = word2vec_model.wv.vocab.keys()
  vector_list = word2vec_model.wv.vectors.tolist()
  data_tuples = list(zip(productID_list,vector_list))
  product_ids_and_vectors = pd.DataFrame(data_tuples, columns=['Product_ID','Vectors'])

  layer.log({"Dataset Description": "Product Vectors (Embeddings) generated by word2vec algorithm."})

  return product_ids_and_vectors

def plot_cluster_distribution(kmeans_model):
  import matplotlib.pyplot as plt

  plt.hist(kmeans_model.labels_, rwidth=0.7)
  plt.ylabel("Number of Products")
  plt.xlabel("Cluster No")

  # Layer logs the plot (figure)
  fig = plt.gcf()
  layer.log({"Product Distribution over Clusters": fig})

  # clear all plots and figures from memory
  plt.figure().clear()
  plt.close()
  plt.cla()
  plt.clf()


def plot_cluster_scatter(product_vectors):
  import matplotlib.pyplot as plt
  from sklearn.decomposition import PCA
  from sklearn.cluster import KMeans

  pca = PCA(n_components=2)
  two_dimensions_vectors = pca.fit_transform(product_vectors)
  
  kmeans_model = KMeans(n_clusters=10, random_state=0).fit(two_dimensions_vectors)
  label = kmeans_model.fit_predict(two_dimensions_vectors)

  #Getting the Centroids
  centroids = kmeans_model.cluster_centers_
  u_labels = np.unique(kmeans_model.labels_)
  
  #plotting the results:
  for i in u_labels:
      plt.scatter(two_dimensions_vectors[label == i , 0] , two_dimensions_vectors[label == i , 1] , label = i)
  plt.scatter(centroids[:,0] , centroids[:,1] , s = 80, color = 'k')
  plt.legend(bbox_to_anchor =(1, 1))
  

  # Layer logs the plot (figure)
  fig = plt.gcf()
  layer.log({"Clusters Formation Scatter Plot": fig})

  # clear all plots and figures from memory
  plt.figure().clear()
  plt.close()
  plt.cla()
  plt.clf()

@model("clustering_model",dependencies=[Dataset("product_ids_and_vectors")])
def fit_kmeans():
  import gensim
  from gensim.models import Word2Vec
  from sklearn.cluster import KMeans
  import matplotlib.pyplot as plt
  
  # Get product vectors from Word2Vec
  product_ids_and_vectors = layer.get_dataset("product_ids_and_vectors").to_pandas()
  array_product_vectors = np.array(product_ids_and_vectors["Vectors"].values.tolist())

  # Fit K-Means algorithm on those embeddings
  kmeans_model = KMeans(n_clusters=10, random_state=0).fit(array_product_vectors)

  # Cluster Distribution Plot
  plot_cluster_distribution(kmeans_model)

  # Cluster Scatter Plot
  plot_cluster_scatter(array_product_vectors)
  
  return kmeans_model

@dataset("final_product_clusters", dependencies=[Model("clustering_model"), Dataset("product_ids_and_vectors")])
def save_final_product_clusters():
  model = layer.get_model("clustering_model").get_train()
  
  product_ids_and_vectors = layer.get_dataset("product_ids_and_vectors").to_pandas()
  array_product_vectors = np.array(product_ids_and_vectors["Vectors"].values.tolist())

  assigned_cluster_no = model.fit_predict(array_product_vectors).tolist()

  df["Cluster_No"] = assigned_cluster_no
  cluster_members_df = df[["Product_ID","Cluster_No"]].groupby("Cluster_No")['Product_ID'].apply(list).to_frame().reset_index().rename(columns={'Product_ID': 'Cluster_Member_List'})

  layer.log({"Dataset Description": "Product Clusters Members List"})

  return cluster_members_df


# **Run your project on Layer in local mode**

To run your project locally on Layer, just call your functions in the correct order and Layer will log and save all the data you asked for automatically.

If you would like to run it in remote mode, put all you functions into the layer.run(), then you are good to go! It will run these functions in its correct order on Layer Infra remotely with respect to dependencies.

In [ ]:
## LAYER LOCAL MODE
raw_session_based_clickstream_data()
generate_sequential_products()
create_product_embeddings()
fit_kmeans()
save_final_product_clusters()


## LAYER REMOTE MODE
# layer.run([raw_session_based_clickstream_data,
#            generate_sequential_products,
#            create_product_embeddings, 
#            fit_kmeans,
#            save_final_product_clusters],debug=True)

# **Let's produce some recommendations for a given product!**

In [45]:
# Fetch the K-Means Model from Layer
kmeans_model = layer.get_model("clustering_model").get_train()

# Fetch product vectors (embeddings) dataset from Layer
product_ids_and_vectors = layer.get_dataset("product_ids_and_vectors").to_pandas()

# Product ID to generate recommendations for - You could try different product IDs in the data such as A16, C17, P12 etc.
product_id = "A13"

# Get Vector (Embedding) array of the given product
vector_array = np.array(product_ids_and_vectors[product_ids_and_vectors["Product_ID"]==product_id]["Vectors"].tolist())

# Get cluster number for the given product assigned by the model
cluster_no = model.predict(vector_array)[0]

# Fetch final clusters members list dataset from Layer
final_product_clusters = layer.get_dataset("final_product_clusters").to_pandas()

# Get members list of the cluster that the given product is assigned to 
cluster_members_list = final_product_clusters[final_product_clusters['Cluster_No']==cluster_no]['Cluster_Member_List'].iloc[0].tolist()

# Randomly select 5 product recommendations from the cluster members excluding the given product
from random import sample
cluster_members_list.remove(product_id)
five_product_recommendations = sample(cluster_members_list, 5)

print("5 Similar Product Recommendations for {}: ".format(product_id),five_product_recommendations)

5 Similar Product Recommendations for A13:  ['C17', 'P60', 'C44', 'P56', 'A6']
